In [2]:
import csv
import numpy as np
from datetime import datetime as dt

In [37]:
timeformat = ["60+0", "180+0", "300+0", "600+0", "180+2"]
filegame_count = [0] * len(timeformat)
file_max_count = 1000000
fileindex = [1] * len(timeformat)
f_list = [] * len(timeformat)
wr_list = [] * len(timeformat)
for i in range(0, len(timeformat)):
    f_list.append(open(timeformat[i] +"_"+ str(fileindex[i]) + '.csv', mode = 'a+', newline = ''))
    wr_list.append(csv.writer(f_list[i], delimiter = ','))
wr_header = ["Result", "WhiteElo", "BlackElo", "TimeControl", "Termination",
                     "Color", "MoveNum", "Move", "Type", "Eval", "EvalDiff", "ForcedMate", "Time", "TimeSpent"]

In [52]:
readpgn = open("E:\lichess_db_standard_rated_2023-04.pgn")
count = 1
max = 101706224
keywords = ['Result', 'UTCDate', 'UTCTime', 'WhiteElo', 'BlackElo',
            'WhiteRatingDiff', 'BlackRatingDiff', 'ECO', 'TimeControl', 'Termination', 'Evaluation']
clk_str = "%clk"
eval_str = "%eval"
test_int = 0
while(count < max):
    notcomplete = True
    line = [""] * 11
    store_rl = ""
    while(notcomplete):
        rl = readpgn.readline()
        rl = rl[1:-2]
        header = rl.split(" ", 1)[0]
        s = ""
        if(header in keywords):
            s = rl.split(" ", 1)[1]
            s = s[1:-1]
            ind = keywords.index(header)
            line[ind] = s
        if(header == "Result"):
            if s == "1-0":
                line[0] = 1
            elif "1/2" in s:
                line[0] = 0.5
            else:
                line[0] = 0
        if(header == "Termination"):
            line[10] = "No"
            if(s == "Abandoned"):
                line[10] = "No"
            else:
                rl = readpgn.readline()
                rl = readpgn.readline()
                if clk_str in rl:
                    if eval_str in rl:
                        line[10] = "Yes"
                        store_rl = rl
                    else:
                        line[10] = "No"
            notcomplete = False
    if(line[10] == "Yes" and line[8] in timeformat):
        ind = timeformat.index(line[8])
        if(filegame_count[ind] > 100000):
            fileindex[ind]+= 1
            filegame_count[ind] = 0
            f_list[ind] = open(timeformat[ind] +"_"+ str(fileindex[ind]) + '.csv', mode = 'a+', newline = '')
            wr_list[ind] = csv.writer(f_list[ind], delimiter = ',')
        
        store_rl = store_rl.translate({ord(i): None for i in '{}[]'})
        text_spl = store_rl.split(" ")
        text_spl = [i for i in text_spl if i not in  ["", "%clk", "%eval"]]
        text_spl = text_spl[:-1]
        count_spl = 0
        prev_eval = "0"
        white_time = int(line[8].split("+")[0])
        black_time = white_timeㅋ
        increment = int(line[8].split("+")[1])
        kekw = len(text_spl) % 4
        
        while(count_spl < len(text_spl)):
            # Write Existing Headers
            wr_line = [""] * len(wr_header)
            wr_line[0] = line[0]
            wr_line[1] = line[3]
            wr_line[2] = line[4]
            wr_line[3] = line[8]
            wr_line[4] = line[9]
            # Designate 5 = color 6 = movenum 7 = move 8 = movetype
            if("..." in text_spl[count_spl]):
                wr_line[5] = "Black"
            else: 
                wr_line[5] = "White"
            wr_line[6] = text_spl[count_spl].strip(".")
            count_spl += 1
            
            wr_line[7] = text_spl[count_spl].strip("?!")
            if("??" in text_spl[count_spl]):
                wr_line[8] = "Blunder"
            elif("!!" in text_spl[count_spl]):
                wr_line[8] = "Brilliant"
            elif("?!" in text_spl[count_spl]):
                wr_line[8] = "Dubious"
            elif("!?" in text_spl[count_spl]):
                wr_line[8] = "Interesting"
            elif("!" in text_spl[count_spl]):
                wr_line[8] = "Good"
            elif("?" in text_spl[count_spl]):
                wr_line[8] = "Mistake"
            else:
                wr_line[8] = "Normal"
            if("\n" in text_spl[count_spl]):
                break
            count_spl+=1
            # Eval / EvalDiff / ForcedMate
            if("#" in wr_line[7]):
                sign = 1
                if(wr_line[5] == "Black"):
                    sign = -1
                eval = 99 * sign
                eval_diff = 0
                wr_line[11] = sign
            elif(":" in text_spl[count_spl]):
                wr_line[9] = 0
                wr_line[10] = 0
                wr_line[11] = 0
                prev_eval = 0
            else:
                eval = text_spl[count_spl].strip("]")
                wr_line[9] = eval
                if("#" in eval):
                    wr_line[11] = np.sign(int(eval.strip("#")))
                    if("#" in prev_eval):
                        wr_line[10] = 0
                    else:
                        wr_line[10] = wr_line[11] * 99
                elif("#" in prev_eval):
                    wr_line[11] = 0
                    wr_line[10] = -99 * np.sign(int(prev_eval.strip("#")))
                else:
                    wr_line[10] = np.round(float(wr_line[9]) - float(prev_eval), 2)
                    wr_line[11] = 0
                prev_eval = str(wr_line[9])
                if(wr_line[5] == "Black"):
                    wr_line[10] = -1 * wr_line[10]
                count_spl+=1
            # Time / Time Spent
            pt = dt.strptime(text_spl[count_spl].strip("]"), '%H:%M:%S')
            wr_line[12] = pt.hour * 3600 + pt.minute * 60 + pt.second
            if(wr_line[5] == "White"):
                wr_line[13] = white_time - wr_line[12] + increment
                white_time = wr_line[12]
            else:
                wr_line[13] = black_time - wr_line[12] + increment
                black_time = wr_line[12]
            
            wr_list[ind].writerow(wr_line)
            filegame_count[ind] += 1
            count_spl += 1
        
    count += 1
readpgn.close()

IndexError: list index out of range